In [1]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
!pip install dspy-ai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

In [21]:
from dotenv import load_dotenv
from typing import List, Union
import matplotlib.pyplot as plt
import os, re, time, random
import networkx as nx
import pandas as pd
import numpy as np
import nltk
from nltk import sent_tokenize

In [20]:
import dspy

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Server/python/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2664484/3968466993.py", line 1, in <module>
    import dspy
  File "/home/cggamella/.local/lib/python3.10/site-packages/dspy/__init__.py", line 1, in <module>
    import dsp
  File "/home/cggamella/.local/lib/python3.10/site-packages/dsp/__init__.py", line 1, in <module>
    from .modules import *
  File "/home/cggamella/.local/lib/python3.10/site-packages/dsp/modules/__init__.py", line 2, in <module>
    from .azure_openai import AzureOpenAI
  File "/home/cggamella/.local/lib/python3.10/site-packages/dsp/modules/azure_openai.py", line 15, in <module>
    import openai
  File "/home/cggamella/.local/lib/python3.10/site-packages/openai/__init__.py", line 8, in <module>
    from . import types
  File "/home/cggamella/.local/lib/python3.10/site-packages/openai/

In [22]:
#Imports
import pathlib

import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Server/python/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2664484/734980704.py", line 4, in <module>
    import dspy
  File "/home/cggamella/.local/lib/python3.10/site-packages/dspy/__init__.py", line 1, in <module>
    import dsp
  File "/home/cggamella/.local/lib/python3.10/site-packages/dsp/__init__.py", line 1, in <module>
    from .modules import *
  File "/home/cggamella/.local/lib/python3.10/site-packages/dsp/modules/__init__.py", line 2, in <module>
    from .azure_openai import AzureOpenAI
  File "/home/cggamella/.local/lib/python3.10/site-packages/dsp/modules/azure_openai.py", line 15, in <module>
    import openai
  File "/home/cggamella/.local/lib/python3.10/site-packages/openai/__init__.py", line 8, in <module>
    from . import types
  File "/home/cggamella/.local/lib/python3.10/site-packages/openai/t

In [4]:
# Para comprobar la conexión al servidor LLaMA
# Si estás en kumo, como el servicio está en kumo01
#!curl -X POST http://kumo01:8090/generate -d '{"prompt": "Hello"}'
# Si estás en kumo01, como el servicio está en kumo01
!curl -X POST http://127.0.0.0:8090/generate -d '{"prompt": "Hello"}'

Expected request with `Content-Type: application/json`

In [17]:
###########
#   LLM   #
###########
path_env = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

lm = dspy.HFClientTGI(model="meta-llama/meta-llama-3-8b-instruct", port=8090, url="http://127.0.0.0") #"meta-llama/Meta-Llama-3-8B "
#lm = dspy.OpenAI(model="gpt-3.5-turbo")# "gpt-4o-2024-05-13")

path_env = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("HUGGINGFACE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = api_key

dspy.settings.configure(lm=lm, temperature = 0)

/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env


NameError: name 'dspy' is not defined

# Detección de acrónimos con dspy  (Etapa 1)

In [4]:
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/files/acronyms_paper.xlsx'
df = pd.read_excel(ruta_archivo)

In [5]:
df.columns

Index(['acronyms', 'manual_expanded_acronyms', 'text', 'expansion', 'context',
       'idDB'],
      dtype='object')

In [6]:
df.head(2)

,acronyms,manual_expanded_acronyms,text,expansion,context,idDB
0,ONG,Organización no gubernamental,la ong ha lanzado una campaña de concienciació...,NaN,NaN,NaN
1,PCAP,Pliego de cláusulas administrativas particulares,el contrato de obra pública se regirá por el p...,NaN,NaN,NaN


In [ ]:
'''
    TEXTO: La reforma del sistema tributario ha traído cambios significativos en el IVA,
    afectando consumidores. Las pyme han mostrado preocupación por el aumento de la cargafiscal. Por otro lado, el gobierno ha ajustado las tasas del IRPF para aliviar la presión
    sobre las rentas más bajas. Según los últimos informes, el PIB del país ha crecido un 2% en el último trimestre.
    ACRONIMOS:IVA, pyme, IRPF, PIB
    
    TEXTO: lote 3 del acuerdo marco para la ejecución de las obras de urbanización accesorias en actuaciones edificatorias
    con cargo a particulares y en los sistemas de gestión por cooperación y expropiación.
    ACRONIMOS:/
    
    TEXTO: obras de accesorios en actuaciones vehiculares espaciales de la NASA
    ACRONIMOS: NASA
'''

In [855]:
class AcronymDetector(dspy.Signature):
    """
    Extrae los acrónimos, siglas y abreviaturas del texto.
    """
    TEXTO = dspy.InputField(desc = "Texto sobre el que se extraen acrónimos, siglas o abreviaturas.")
    ACRONIMOS = dspy.OutputField(desc = "Palabras que son acrónimos, siglas o abreviaturas que aparecen en el texto.")

class AcronymDetectorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        #self.generator = dspy.ChainOfThought(AcronymDetector)
        self.generator = dspy.Predict(AcronymDetector)
        
        self.no_acronym_variations = [
            '/', '/ (no hay acrónimos)','','${TEXTO}','${ACRONIMOS}','/ (no hay acronimos)',
            '(no hay acronimos)', 'no hay acronimos',
            '/ (No acronyms present in the document)',
            '/ (not present in the document)', "'/'", 'N/A', '/.'
        ]
    
    def _process_output(self, texto):

        if texto in self.no_acronym_variations or "N_A" in texto or "N/A" in texto or "NA" in texto:
            return "/"
        #if len(texto) > 40:
        #    return "/"
        else:
            return texto

    def forward(self, texto):
        response = self.generator(TEXTO=texto)
        print("La respuesta es:",response)
        if len(texto) == len(response.ACRONIMOS) or texto == response.ACRONIMOS:
            return dspy.Prediction(ACRONIMOS='/')
        else:   
            return dspy.Prediction(ACRONIMOS=self._process_output(response.ACRONIMOS))

In [856]:
texto = 'Elaboración del estudio en la pib ambiental de la zona costera de Monpas y de los estuarios de los ríos Oiartzun y Urumea'
module = AcronymDetectorModule()
module(texto)

La respuesta es: Prediction(
    ACRONIMOS='pib, Monpas, Oiartzun, Urumea'
)


Prediction(
    ACRONIMOS='pib, Monpas, Oiartzun, Urumea'
)

In [701]:
class AcronymValidator(dspy.Signature):
    """
    Verifica si la palabra es un acrónimo, sigla o abreviatura 'true', o 'false' por el contrario. 
    """
    ACRONIMO = dspy.InputField(desc = "Acrónimo, sigla o abreviatura pendiente de verificar")
    CONTEXTO = dspy.InputField()
    RESULTADO = dspy.OutputField()

class AcronymValidatorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.Predict(AcronymValidator)

    def forward(self, acronimo, contexto):
        # Llamar al modelo con cada acrónimo y contexto asociado
        response = self.generator(ACRONIMO=acronimo, CONTEXTO=contexto)
        # Imprimir la respuesta para verificar su estructura
        print("Response:", response.RESULTADO)
        
        return dspy.Prediction(RESULTADO = response.RESULTADO)

In [702]:
acronimo = 'elaboración'
contexto = 'Elaboración del estudio en la pib ambiental de la zona costera de Monpas y de los estuarios de los ríos Oiartzun y Urumea'

module = AcronymValidatorModule()
module(acronimo, contexto)

Response: true


Prediction(
    RESULTADO='true'
)

In [459]:
def filter_acronyms_in_text(text, acronyms):
    """
    Filters and returns a list of acronyms that are present in the given text.

    acronyms: List of acronyms to check.
    text: Text in which to search for the acronyms.
    return: List of acronyms that are present in the text.
    """
    words_in_text = text.split()
    # Filtramos los acrónimos que coinciden exactamente con alguna palabra en el texto
    filtered_acronyms = [acronym for acronym in acronyms if acronym.lower() in words_in_text]

    return filtered_acronyms

def filter_companies(acronyms):
    """
    Filters out acronyms that contain any variant of 'SL', 'SLU', or 'SA',
    regardless of case, with or without periods.

    acronyms: List of acronyms to filter.
    return: List of acronyms excluding those with 'SL', 'SLU', or 'SA' variants.
    """
    
    # Define the regular expression pattern for variants
    pattern = re.compile(
        r'\b(?:S\.?L\.?(?:U\.?)?|S\.?A\.?)\b',
        re.IGNORECASE
    )
    
    # Filter out acronyms matching the pattern
    filtered_acronyms = [acronym for acronym in acronyms if not pattern.search(acronym)]
    
    return filtered_acronyms

def filter_items_and_acronyms(items):
    """
    Filters items from a list based on the following criteria:
    - Items that have more than 10 characters
    - Items that are composed of more than 2 words
    - Items that have more than three numeric characters

    items: List of strings to filter.
    return: List of strings that meet the criteria.
    """
    
    # Use filter items based on all criteria
    filtered_items = [
        item for item in items
        if len(item) <= 10
        and len(item.split()) <= 2
        and sum(c.isdigit() for c in item) <= 3
    ]
    
    return filtered_items

In [310]:
'''
lista = ['TEVA PHARMA S.LU.', 'S.L.U.', 'S.L.','S.L.U.','hola']
lista = filter_companies(lista)
lista
'''

['hola']

In [461]:
'''
# Example usage
acronyms = ['vp']
text = "Refuerzo de firme vp en la VP 4013 Melgar de Arriba a Villacarralón. vp Tramo: Santervás a Villacarralón"
result = filter_acronyms_in_text(text, acronyms)
result
'''

['vp']

In [ ]:
'''
import time

# Record the start time
start_time = time.time()
    
# Crear una instancia del módulo
my_acronym_validator = AcronymValidationModule()

df_esp['Acronyms Validated'] = ''
df_esp['Not Acronyms'] = ''

# Validate if acronyms detected are acronyms
for index, row in df_esp.iterrows():
    print(f"Processing row {index}: {row['detected_acronyms']}")
    print("EL TEXTO QUE SE ANALIZA ES:", row['text'])
    # Split acronym list convert str to list
    detected_acronyms = [acronym.strip() for acronym in row['detected_acronyms'].split(',')]
    # Filter if detected acronym its '/','','n/a','none', etc (all possible answer of LLM)
    detected_acronyms = [acronym.strip() for acronym in detected_acronyms if acronym.strip() not in ['', 'n/a', 'none', 'ninguno','/', '(no hay acrónimos)']]
    print("Los detected (1)", detected_acronyms)
    # If detected acronyms list is empty, break
    if not detected_acronyms:
        print(f"No acronyms detected in row {index}, continue the loop.")
        continue
           
    # Keep only detected acronym items that has less than 3 numbers, less than 10 characters, less than 2 words --> its not acronym
    detected_acronyms = filter_items_and_acronyms(detected_acronyms)
    # If sl, slu, sa and his versions its detected remove item from list -> its a company name not acronym
    detected_acronyms = filter_companies(detected_acronyms)
    print("Los detected (2)", detected_acronyms)
    # Auxiliar function to check if the detected acr its contained in the text -> avoid hallucinations
    detected_acronyms = filter_acronyms_in_text(row['text'],detected_acronyms)
    print("Los detected (3)", detected_acronyms)   
    # If detected acronyms list is empty, continue
    if not detected_acronyms:
        print(f"No acronyms AFTER FILTERING in row {index}, continue the loop.")
        continue
    
    # List to keep valid acronyms after filtering
    acronyms_validated = []
    not_acronyms = []
    
    for acronym in detected_acronyms:
        try:
            # Validate each acronym individually
            prediction = my_acronym_validator.forward(acronym)
            print(f"Result of acronym '{acronym}': {prediction.RESULTADO}")
            
            # Determinar si el acrónimo es válido o no
            if prediction.RESULTADO.lower() == 'true':  
                acronyms_validated.append(acronym)
            else:
                not_acronyms.append(acronym)
        
        except Exception as e:
            print(f"Error processing acronym '{acronym}' in row {index}: {e}")
    
    # Almacenar los resultados en las columnas del DataFrame
    df_esp.at[index, 'Acronyms Validated'] = ', '.join(acronyms_validated)
    df_esp.at[index, 'Not Acronyms'] = ', '.join(not_acronyms) 

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
total_time = end_time - start_time
print(f"Task for {len(df_esp)} rows its completed in {total_time:.6f} seconds")
'''

In [152]:
#df_esp.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/df_esp_1000_detect.xlsx')

# Anotación manual

In [ ]:
'''
def create_subdataframes(df):
    # Specify the columns to exclude
    columns_to_exclude = ['detected_acronyms', 'Acronyms Validated', 'Not Acronyms', 'idioma']
    
    # Select the first 500 rows and exclude the specified columns
    df_subset = df.iloc[:500].drop(columns=columns_to_exclude, errors='ignore')
    
    # Split the DataFrame into chunks of 100 rows each
    subdataframes = [df_subset.iloc[i:i + 100] for i in range(0, 500, 100)]
    
    return subdataframes

# df_esp = pd.read_csv('your_data.csv')  # Example to load your data

subdataframes = create_subdataframes(df_esp)

# Assigning specific names to each sub-dataframe
df_part1 = subdataframes[0]
df_part2 = subdataframes[1]
df_part3 = subdataframes[2]
df_part4 = subdataframes[3]
df_part5 = subdataframes[4]

for df in [df_part1, df_part2, df_part3, df_part4, df_part5]:
    df['detected_acr'] =''
    
df_part1.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/df_esp_first100.xlsx')
df_part2.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/df_esp_100_200.xlsx')
df_part3.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/df_esp_200_300.xlsx')
df_part4.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/df_esp_300_400.xlsx')
df_part5.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/df_esp_400_500.xlsx')
'''

In [ ]:
'''
# Crear una instancia del módulo
my_acronym_detector = AcronymDetectorModule()

# Lista para almacenar los acrónimos detectados
acronyms_detected = []

# Detectar acrónimos en cada texto del DataFrame
for index, row in df_esp.iterrows():
    print(f"Processing row {index}: {row['text']}")
    prediction = my_acronym_detector.forward(row['text'])
    print(f"Detected acronyms: {prediction.ACRONIMOS}")
    acronyms_detected.append(prediction.ACRONIMOS)
'''

In [149]:
# Guardar los acrónimos detectados
#df_esp['detected_acronyms'] = acronyms_detected

# Nueva forma de hacer chunks

In [197]:
# La modifico incluyendo como parámetro el window_overlap para intanciarla con mayor control y ajuste del final del doc
class Chunker:
    def __init__(self, context_window=3000, max_windows=5, window_overlap = 0.02):
        self.context_window = context_window
        self.max_windows = max_windows
        self.window_overlap = window_overlap

    def __call__(self, paper):
        snippet_idx = 0
        startpos = 0

        while snippet_idx < self.max_windows and len(paper) > startpos:
            endpos = startpos + int(self.context_window * (1.0 + self.window_overlap))
            if endpos > len(paper):
                endpos = len(paper)
            snippet = paper[startpos:endpos]

            next_newline_pos = snippet.rfind('\n')
            if next_newline_pos != -1 and next_newline_pos >= self.context_window // 2:
                snippet = snippet[:next_newline_pos + 1]

            yield snippet_idx, snippet.strip()
            startpos += self.context_window - int(self.context_window * self.window_overlap)
            snippet_idx += 1

In [198]:
chunker = Chunker(context_window=3000, max_windows=100, window_overlap = 0.1)

In [860]:
def process_dataframe(path):
    
    df = pd.read_excel(path) 
    df = df.copy()

    my_acronym_detector = AcronymDetectorModule()
    # Instancia de Chunker
    chunker = Chunker(context_window=3000, max_windows=100, window_overlap = 0.1)
    
    for identifier, row in df.iterrows():
        text = row['text']  # Texto de cada fila
        print(f"PROCESSING ROW WITH TEXT: {row['text']}")
        
        # Reiniciar las listas para cada entrada del DataFrame
        acronyms_validated = []
        not_acronyms = []
        
        # Detectar acrónimos y calcular embeddings para cada chunk
        for id_chunk, chunk in chunker(text):
            print(f"PROCESSING CHUNK {id_chunk}") 

            # Detectar acrónimos en cada chunk
            prediction = my_acronym_detector.forward(chunk)
            acronyms = clean_acronyms(prediction.ACRONIMOS)
            acronyms_list = acronyms.lower().split(',')
            print(f"DETECTED ACR IN CHUNK: {acronyms_list}")
            
            # Funciones auxiliares para filtrar los acrónimos
            detected_acronyms = [acronym.strip() for acronym in acronyms_list]
            detected_acronyms = set(detected_acronyms)
            detected_acronyms = [acronym.strip() for acronym in detected_acronyms if acronym.strip() not in ['/', 'n/a', 'none', 'ninguno', '', '(no hay acrónimos)']]
            print("Los detected (1)", detected_acronyms)
            if not detected_acronyms:
                print(f"No acronyms detected in row {identifier}, continue the loop.")
                df.at[identifier, 'Is there any acronym'] = False
                continue

            detected_acronyms = filter_items_and_acronyms(detected_acronyms)
            detected_acronyms = filter_companies(detected_acronyms)
            print("Los detected (2)", detected_acronyms)
            detected_acronyms = filter_acronyms_in_text(row['text'], detected_acronyms)
            print("Los detected (3)", detected_acronyms)   
            if not detected_acronyms:
                print(f"No acronyms AFTER FILTERING in row {identifier}, continue the loop.")
                df.at[identifier, 'Is there any acronym'] = False
                continue

            for acronym in detected_acronyms:
                df.at[identifier, 'Acronyms Detected(LLM)'] = ', '.join(detected_acronyms)
    return df

In [861]:
df = process_dataframe('/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/fam/df_esp_100_200.xlsx')

PROCESSING ROW WITH TEXT: Servicio de mantenimiento de pyxis y kardex ubicados en el H.U. Santa María del Rosell y H.G.U. Santa Lucía de Cartagena
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='H.U. Hospital Universitario, H.G.U. Hospital General Universitario, pyxis, kardex'
)
DETECTED ACR IN CHUNK: ['h.u. hospital universitario', ' h.g.u. hospital general universitario', ' pyxis', ' kardex']
Los detected (1) ['pyxis', 'h.g.u. hospital general universitario', 'kardex', 'h.u. hospital universitario']
Los detected (2) ['pyxis', 'kardex']
Los detected (3) ['pyxis', 'kardex']
PROCESSING ROW WITH TEXT: Suministro eléctrico para edificios de uso administrativo Aseinfante de Murcia y Foro de Cartagena, adscritos a la Consejería de Hacienda y Administraciones Públicas de la CARM.
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='Aseinfante, CARM, Foro de Cartagena, Consejería de Hacienda y Administraciones Públicas, CARM.'
)
DETECTED ACR IN CHUNK: ['aseinfante', '

La respuesta es: Prediction(
    ACRONIMOS='"Mtto. integral, TAc, GCD07x3084, Toshiba, Radiología Gral, HCUVA"'
)
DETECTED ACR IN CHUNK: ['"mtto. integral', ' tac', ' gcd07x3084', ' toshiba', ' radiología gral', ' hcuva"']
Los detected (1) ['tac', '"mtto. integral', 'toshiba', 'gcd07x3084', 'hcuva"', 'radiología gral']
Los detected (2) ['tac', 'toshiba', 'hcuva"']
Los detected (3) []
No acronyms AFTER FILTERING in row 14, continue the loop.
PROCESSING ROW WITH TEXT: Construcción de intersección de la carretera RM-714 con camino cañada garcia P.K. 57+700 P.K. de Cehegin
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='RM-714, P.K.'
)
DETECTED ACR IN CHUNK: ['rm-714', ' p.k.']
Los detected (1) ['rm-714', 'p.k.']
Los detected (2) ['rm-714', 'p.k.']
Los detected (3) []
No acronyms AFTER FILTERING in row 15, continue the loop.
PROCESSING ROW WITH TEXT: Gestión del Programa Europa Joven Madrid y del Programa Xarxa Formación Profesional
PROCESSING CHUNK 0
La respuesta es: Predic

La respuesta es: Prediction(
    ACRONIMOS='Atención a personas mayores dependientes (residencia y centro de día) de Peñuelas, Madrid'
)
DETECTED ACR IN CHUNK: ['atención a personas mayores dependientes de peñuelas', ' madrid']
Los detected (1) ['madrid', 'atención a personas mayores dependientes de peñuelas']
Los detected (2) ['madrid']
Los detected (3) []
No acronyms AFTER FILTERING in row 29, continue the loop.
PROCESSING ROW WITH TEXT: Servicios de transporte de paquetería del Organismo Autónomo Boletín Oficial de la Comunidad de Madrid
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='OABOCM'
)
DETECTED ACR IN CHUNK: ['oabocm']
Los detected (1) ['oabocm']
Los detected (2) ['oabocm']
Los detected (3) []
No acronyms AFTER FILTERING in row 30, continue the loop.
PROCESSING ROW WITH TEXT: El objeto de este contrato es la prestación de los servicios para una promoción de alimentos de Castilla y León en un centro de distribución Comercial con formato supermercado línea gour

La respuesta es: Prediction(
    ACRONIMOS='Nadal, Santa Rita, San Roque, fiestas, orquestas'
)
DETECTED ACR IN CHUNK: ['nadal', ' santa rita', ' san roque', ' fiestas', ' orquestas']
Los detected (1) ['orquestas', 'nadal', 'san roque', 'fiestas', 'santa rita']
Los detected (2) ['orquestas', 'nadal', 'san roque', 'fiestas', 'santa rita']
Los detected (3) ['orquestas', 'fiestas']
PROCESSING ROW WITH TEXT: Elaboración del estudio ambiental de la zona costera de Monpas y de los estuarios de los ríos Oiartzun y Urumea
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='EZA, EIA, EIAZ, EIAZC, EIAZC-M, EIAZC-MO, EIAZC-MO-UR, EIAZC-MO-URU, EIAZC-MO-URUO, EIAZC-MO-URUO-I, EIAZC-M'
)
DETECTED ACR IN CHUNK: ['eza', ' eia', ' eiaz', ' eiazc', ' eiazc-m', ' eiazc-mo', ' eiazc-mo-ur', ' eiazc-mo-uru', ' eiazc-mo-uruo', ' eiazc-mo-uruo-i', ' eiazc-m']
Los detected (1) ['eiazc-mo-ur', 'eiazc-mo-uru', 'eza', 'eiazc-mo-uruo', 'eiazc-m', 'eiazc', 'eiazc-mo-uruo-i', 'eia', 'eiaz', 'eiazc-mo']


La respuesta es: Prediction(
    ACRONIMOS='C.C. = Centro Comercial'
)
DETECTED ACR IN CHUNK: ['c.c. = centro comercial']
Los detected (1) ['c.c. = centro comercial']
Los detected (2) []
Los detected (3) []
No acronyms AFTER FILTERING in row 62, continue the loop.
PROCESSING ROW WITH TEXT: Contrato basado nº 25 del Lote 3 del Acuerdo marco de servicios de “Trabajos de apoyo a la redacción de proyectos de ejecución de obras para la Dirección General de Patrimonio (4 lotes)"
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='Lote 3, Acuerdo marco, Trabajos de apoyo, redacción de proyectos, Dirección General de Patrimonio, 4 lotes'
)
DETECTED ACR IN CHUNK: ['lote 3', ' acuerdo marco', ' trabajos de apoyo', ' redacción de proyectos', ' dirección general de patrimonio', ' 4 lotes']
Los detected (1) ['trabajos de apoyo', 'lote 3', 'acuerdo marco', '4 lotes', 'redacción de proyectos', 'dirección general de patrimonio']
Los detected (2) ['lote 3', '4 lotes']
Los detected (3) []
No 

La respuesta es: Prediction(
    ACRONIMOS='DGEPC, Ayuntamiento de Madrid, DGEP, DGEPAC, DGEPACM, DGEPACMAD, DGEPACMAD, DGEPACMAD, DGEPACMAD, DGEPACMAD, DGEPACMAD, DGEPACMAD, DGEPACMAD, DGEPACMAD'
)
DETECTED ACR IN CHUNK: ['dgepc', ' ayuntamiento de madrid', ' dgep', ' dgepac', ' dgepacm', ' dgepacmad', ' dgepacmad', ' dgepacmad', ' dgepacmad', ' dgepacmad', ' dgepacmad', ' dgepacmad', ' dgepacmad', ' dgepacmad']
Los detected (1) ['dgepc', 'ayuntamiento de madrid', 'dgepac', 'dgep', 'dgepacm', 'dgepacmad']
Los detected (2) ['dgepc', 'dgepac', 'dgep', 'dgepacm', 'dgepacmad']
Los detected (3) []
No acronyms AFTER FILTERING in row 76, continue the loop.
PROCESSING ROW WITH TEXT: Suministro de material de oficina y papel de impresión y escritura con criterios medioambientales
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='Suministro de material de oficina y papel de impresión y escritura con criterios medioambientales'
)
DETECTED ACR IN CHUNK: ['suministro de material de of

La respuesta es: Prediction(
    ACRONIMOS='AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC, AOP, AOC,'
)
DETECTED ACR IN CHUNK: ['aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', ' aop', ' aoc', '']
Los detected (1) ['aop', 'aoc']
Los detected (2) ['aop', 'aoc']
Los detected (3) []
No acronyms AFTER FILTERING in row 92, continue the loop.
PROCESSING ROW WITH TEXT: Servicios para el seguimiento del proceso de sostenibilidad de Errenteria: Agenda Local 21 y Secretaría Técnica del Consejo Asesor de Medio Ambiente (CAMA) 
PROCESSING CHUNK 0
La respuesta es: Prediction(
    ACRONIMOS='Agenda Local 21, CAMA'
)
DETECTED ACR IN CHUNK: ['agenda local 21', ' cama']
Los detected (1) ['cama', 'agenda local 21']
Los detected (2) ['cama']
Los detected (3) []
No acronyms AFTER FILTERING in row 93, contin

In [863]:
df[3:7]

,zip,file name,entry,text,detected_acr,acr_des,Acronyms Validated,Not Acronyms,Is there any acronym
3,NaN,NaN,93,Contrato mixto de obras: redacción de proyecto...,/,/,,,False
4,NaN,NaN,92,Contrato mixto de obras: redacción de proyecto...,/,/,,,False
5,NaN,NaN,90,obras de mejora de la instalación eléctrica en...,IES,instituto de educación secundaria,,,False
6,NaN,NaN,89,"La contratación, por procedimiento abierto, de...",C.E.E.,comunidad económica europea,,,False


In [ ]:
'''
# Normalizar el campo textual para evitar problemas
df['text'] = df['text'].str.lower()

documents = []

for idx, row in df.iterrows():
    example_document = row['text']
    print("EXAMPLE DOC:\n",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial"})
    print("El DOC es:\n",doc)

    # Dividir el documento en fragmentos usando la instancia de Chunker
    for _, chunk in chunker(doc.page_content):
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)
        
# Normalizar el campo textual para evitar problemas
df['text'] = df['text'].str.lower()
# Crear el text splitter, [chunk_size: #caracteres de cada chunk];
#[chunk_overlap: #caracteres solapan entre chunks para no perder info.]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

# Crear una lista para almacenar los documentos
documents = []
# Procesar cada fila como un documento separado
for idx, row in df.iterrows():
    example_document = row['text']
    print("el example doc es:",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial"}) #, "identifier": row['identifier']})
    print("El doc es:",doc)
    # Dividir el documento en fragmentos
    chunks = text_splitter.split_text(doc.page_content)
    print("Los chunks son:",chunks)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)
'''

# Create database (old)

In [ ]:
'''
# Embed and store the texts
path_to_index = '/export/usuarios_ml4ds/cggamella/RAG_tool'
# Supplying a persist_directory will store the embeddings on disk
persist_directory = (pathlib.Path(path_to_index) / 'db_POC_acronyms').as_posix()
'''

In [ ]:
'''
# Calcular los embeddings con un modelo de uso libre de HuggingFace
# Using HuggingFace models
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="HUGGINGFACE_API_KEY",
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name)

start = time.time()
# Almacenar los fragmentos en Chroma
# Se extrae el contenido (page_content).El contenido se pasa al modelo de embeddings.
# El vector resultante se almacena en la base de datos junto con los metadatos(índices).

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=model,
    persist_directory=persist_directory
)
end = time.time()
print(f"Total time is {end - start} seconds")
'''

### Create Chroma db without Langchain

In [192]:
import chromadb

path_to_index = '/export/usuarios_ml4ds/cggamella/RAG_tool/example'

In [193]:
class CustomEmbeddings():

    def __init__(self, model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        self.model = SentenceTransformer(model)
        
    def encode(self, texts):
        return self.model.encode(texts)

    def __call__(self, input):
        return self.encode(input)
    
embedding_model = CustomEmbeddings()
client = chromadb.PersistentClient(path=path_to_index)

In [305]:
#Collection for dataset in '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/DESCARGAS_ENTREGABLES/outsiders.parquet'
collection = client.get_or_create_collection(name="test_3", embedding_function=embedding_model)

In [ ]:
'''
results = collection.query(
    query_texts = ["El ceip es un buen sitio para estudiar"],
    n_results = 2,
    include = ['distances','embeddings', 'documents','metadatas']
)
results
'''

In [235]:
def clean_acronyms(text):
    # Remove '(..text..) inside parenthesis'
    cleaned_text = re.sub(r'\s*\([^)]*\)', '', text)
    return cleaned_text.strip()

In [ ]:
collection.peek(50) # returns a list of the first items in the collection

In [244]:
collection.count() # returns the number of items in the collection

0

In [202]:
chunker = Chunker(context_window=3000, max_windows=100, window_overlap = 0.1)

In [203]:
df_esp.head(2)

,text,idioma,identifier
0,Refuerzo de Firme en la VP 3001 Renedo de Esgu...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
1,Refuerzo de Firme en la VP 6603 Mota del Marqu...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...


In [204]:
'''
def addVectorDataToDb(df, chunker) -> None:
    embeddings: list = []
    metadatas: list = []
    documents: list = []
    ids: list = []
    
    df['text'] = df['text'].str.lower() 
    
    try:
        for idx, row in df.iterrows():
            # Obtenemos el 'identifier' de la fila actual
            identifier = row['identifier']
            # divide in chunks
            for id_chunk, chunk in chunker(row.text):
                embeddings.append(embedding_model.encode(chunk).tolist())
                #metadatas.append(f"{str(idx)}_{str(id_chunk)}")
                documents.append(chunk)
                ids.append(f"{str(identifier)}_{str(id_chunk)}")
        collection.add(
            embeddings=embeddings,
            #metadatas=metadatas,
            documents=documents,
            ids=ids
        )
        print("Data added to collection")
    except Exception as e:
        print("Add data to db failed : ", e)
'''

In [205]:
addVectorDataToDb(df_esp, chunker)

Data added to collection


In [ ]:
documents = collection.get(include = ['embeddings', 'documents','metadatas'])
documents

In [207]:
documents['ids'][0:3]

['PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/10_0',
 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/115_0',
 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/116_0']

In [209]:
df_esp

,text,idioma,identifier
0,refuerzo de firme en la vp 3001 renedo de esgu...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
1,refuerzo de firme en la vp 6603 mota del marqu...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
2,refuerzo de firme en la vp 4013 melgar de arri...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
3,desarrollo del programa de intervención socioe...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
4,stand de euskadi en fitur y sus posibles adap...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
...,...,...,...
995,servicio de comunicación de las actividades de...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
996,servicio para la redacción del estudio hidroge...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
997,servicio del programa de intervención social c...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
998,programa municipal denominado punto de informa...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...


In [506]:
my_acronym_detector = AcronymDetectorModule()
my_acronym_validator = AcronymValidatorModule()

# 1st Fill in database with info provided in the dataframe.
def process_dataframe_and_update_db(df, collection): 
    
    df = df.copy()
    
    df['Acronyms Validated'] = ''
    df['Not Acronyms'] = ''
    df['Is there any acronym'] = True
    
    #If index its numeric, replace index by identifier
    first_index_value = df.index[0][0] if isinstance(df.index, pd.MultiIndex) else df.index[0]
    # Comprobar la longitud y aplicar set_index si corresponde
    if len(first_index_value) < 9:
        df.set_index('identifier', inplace=True)

    for identifier, row in df.iterrows():
        text = row['text'] # text for each tender
        # Unique identifier for each tender
        print(f"PROCESSING ROW WITH TEXT:{row['text']}")
        
        # Reset for each entry in the dataframe
        embeddings: list = []
        metadatas: dict = {}
        documents: list = []
        ids: list = []
            
        # List to keep valid acronyms after filtering
        acronyms_validated = []
        not_acronyms = []
        
        # Detect acronyms of each chunk, Chunk the text, then calculate embeddings
        for id_chunk, chunk in chunker(text):
            print(f"PROCESSING CHUNK {id_chunk}") 
            documents.append(chunk)
            print("CHUNK IS:", documents)
            embeddings.append(embedding_model.encode(chunk).tolist())

            # Detect acronyms of each chunk
            prediction = my_acronym_detector.forward(chunk)
            acronyms = clean_acronyms(prediction.ACRONIMOS)
            acronyms_list = acronyms.lower().split(',')
            print(f"DETECTED ACR IN CHUNK BY LLAMA: {acronyms_list}")
            
            # Auxiliar functions to filter 
            detected_acronyms = [acronym.strip() for acronym in acronyms_list]
            # Remove duplicates
            detected_acronyms = set(detected_acronyms)
            # Filter if detected acronym its '/','','n/a','none', etc (all possible answer of LLM)
            detected_acronyms = [acronym.strip() for acronym in detected_acronyms if acronym.strip() not in ['/', 'n/a', 'none', 'ninguno','', '(no hay acrónimos)']]
            print("Los detected (1)", detected_acronyms)
            # If detected acronyms list is empty, break
            if not detected_acronyms:
                print(f"No acronyms detected in row {identifier}, continue the loop.")
                df.at[identifier, 'Is there any acronym'] = False
                continue
            # Keep only detected acronym items that has < 3 numbers, < 10 characters, < 2 words --> its not acronym
            detected_acronyms = filter_items_and_acronyms(detected_acronyms)
            # If sl, slu, sa and his versions its detected remove item from list -> its a company name not acronym
            detected_acronyms = filter_companies(detected_acronyms)
            print("Los detected (2)", detected_acronyms)
            # Auxiliar function to check if the detected acr its contained in the text -> avoid hallucinations
            detected_acronyms = filter_acronyms_in_text(row['text'],detected_acronyms)
            print("Los detected (3)", detected_acronyms)   
            # If detected acronyms list is empty, continue
            if not detected_acronyms:
                print(f"No acronyms AFTER FILTERING in row {identifier}, continue the loop.")
                df.at[identifier, 'Is there any acronym'] = False
                continue

            for acronym in detected_acronyms:
                try:
                    # Validate each acronym individually with new module
                    prediction = my_acronym_validator.forward(acronym)
                    print(f"VALIDATOR SAYS THAT '{acronym}' IT'S: {prediction.RESULTADO}")

                    # Determinar si el acrónimo es válido o no
                    if prediction.RESULTADO.lower() == 'true':  
                        acronyms_validated.append(acronym)
                    else:
                        not_acronyms.append(acronym)

                except Exception as e:
                    print(f"Error processing acronym '{acronym}' in row {identifier}: {e}")

            # Write result in corresponding df columns
            df.at[identifier, 'Acronyms Validated'] = ', '.join(acronyms_validated)
            df.at[identifier, 'Not Acronyms'] = ', '.join(not_acronyms)

            # Update dataBase for each acronym detected
            for i, acronym in enumerate(acronyms_validated):
                acronym = acronym.strip()
                print("EL ACRONYM ES",acronym)
                new_id = f"{identifier}_{id_chunk}"
                print("El NUEVO ID ES:", new_id)
                print("EL ACRONYM ES:", acronym)
                
                # Load metadata for each detected acronym
                metadata = {
                    'index_aux': new_id, # Id to locate certain acronym and know where does it come from
                    'acr': acronym       # Acronym detected
                }
                                
                collection.add(
                    embeddings=embeddings,  
                    documents=documents,       
                    metadatas=metadata,    
                    ids=new_id             
                )
                print(f"Database loaded for document ID {new_id} with metadata: {metadata}")  
            
    return df

In [507]:
df_esp_updated = process_dataframe_and_update_db(df_esp, collection)

PROCESSING ROW WITH TEXT:refuerzo de firme en la vp 3001 renedo de esgueva a pesquera de duero. tramo: villabañez a pk 12+422
PROCESSING CHUNK 0
CHUNK IS: ['refuerzo de firme en la vp 3001 renedo de esgueva a pesquera de duero. tramo: villabañez a pk 12+422']
DETECTED ACR IN CHUNK BY LLAMA: ['vp', ' pk']
Los detected (1) ['pk', 'vp']
Los detected (2) ['pk', 'vp']
Los detected (3) ['pk', 'vp']
Response: true
VALIDATOR SAYS THAT 'pk' IT'S: true
Response: true
VALIDATOR SAYS THAT 'vp' IT'S: true
EL ACRONYM ES pk
El NUEVO ID ES: PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/453_0
EL ACRONYM ES: pk
Database loaded for document ID PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/453_0 with metadata: {'index_aux': 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/453_0', 'acr': 'pk'}
EL ACRONYM ES vp
El NUEVO ID ES: PlataformasAgregadasSinMenores_2018.zip/P

Response: false
VALIDATOR SAYS THAT 'se' IT'S: false
Response: false
VALIDATOR SAYS THAT 'a-6' IT'S: false
Response: false
VALIDATOR SAYS THAT 'lp' IT'S: false
EL ACRONYM ES vp
El NUEVO ID ES: PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/452_0
EL ACRONYM ES: vp
Database loaded for document ID PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/452_0 with metadata: {'index_aux': 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/452_0', 'acr': 'vp'}
PROCESSING ROW WITH TEXT:refuerzo de firme en la vp 4013 melgar de arriba a villacarralón. tramo: santervás a villacarralón
PROCESSING CHUNK 0
CHUNK IS: ['refuerzo de firme en la vp 4013 melgar de arriba a villacarralón. tramo: santervás a villacarralón']
DETECTED ACR IN CHUNK BY LLAMA: ['vp: vía pecuaria', ' melgar de arriba: melgar de arriba', ' villacarralón: villacarralón', ' santervás: santervás', ' tram

DETECTED ACR IN CHUNK BY LLAMA: ['fitur', ' ferias', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' euskadi', ' e']
Los detected (1) ['e', 'ferias', 'euskadi', 'fitur']
Los detected (2) ['e', 'ferias', 'euskadi', 'fitur']
Los detected (3) ['e', 'ferias', 'euskadi', 'fitur']
Response: false
VALIDATOR SAYS THAT 'e' IT'S: false
Response: false
VALIDATOR SAYS THAT 'ferias' IT'S: false
Response: false
VALIDATOR SAYS THAT 'euskadi' IT'S: false
Response: true
VALIDATOR SAYS THAT 'fitur' IT'S: true
EL ACRONYM ES fitur
El NUEVO ID ES: PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/447_0
EL ACRONYM ES: fitur
Database loaded for document ID PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/447_0 with metadata: {'index_aux': 'PlataformasAgregadasSinMenores_2018.zi

DETECTED ACR IN CHUNK BY LLAMA: ['pnsp', ' ct', ' sistema', ' tramitación', ' residuos', ' consejería', ' política', ' territorial', ' sostenibilidad', ' seguridad', ' desarrollo', ' trabajos', ' maintenance', ' sistema', ' tramitación', ' residu']
Los detected (1) ['seguridad', 'pnsp', 'ct', 'sistema', 'sostenibilidad', 'consejería', 'residu', 'tramitación', 'trabajos', 'territorial', 'desarrollo', 'residuos', 'maintenance', 'política']
Los detected (2) ['seguridad', 'pnsp', 'ct', 'sistema', 'consejería', 'residu', 'trabajos', 'desarrollo', 'residuos', 'política']
Los detected (3) ['sistema', 'consejería', 'trabajos', 'desarrollo', 'residuos', 'política']
Response: false
VALIDATOR SAYS THAT 'sistema' IT'S: false
Response: false
VALIDATOR SAYS THAT 'consejería' IT'S: false
Response: false
VALIDATOR SAYS THAT 'trabajos' IT'S: false
Response: false
VALIDATOR SAYS THAT 'desarrollo' IT'S: false
Response: false
VALIDATOR SAYS THAT 'residuos' IT'S: false
Response: false
VALIDATOR SAYS THAT '

DETECTED ACR IN CHUNK BY LLAMA: ['sapg']
Los detected (1) ['sapg']
Los detected (2) ['sapg']
Los detected (3) []
No acronyms AFTER FILTERING in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/398, continue the loop.
PROCESSING ROW WITH TEXT:contrato de servicios para el apoyo a la gestión de la atención personalizada a través de los canales de atención a la ciudadanía de línea madrid
PROCESSING CHUNK 0
CHUNK IS: ['contrato de servicios para el apoyo a la gestión de la atención personalizada a través de los canales de atención a la ciudadanía de línea madrid']
DETECTED ACR IN CHUNK BY LLAMA: ['línea madrid', ' línea', ' madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid', ' línea madrid']
Los detected (1) ['línea madrid', 'línea', 'madrid']
Los detected (2) ['línea', 'madrid']
Los

...

DETECTED ACR IN CHUNK BY LLAMA: ['/']
Los detected (1) []
No acronyms detected in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_5.atom/166, continue the loop.
PROCESSING ROW WITH TEXT:asistencia técnica para soporte experto legal y financiero en el proyecto uia01-060, as-fabrik
PROCESSING CHUNK 0
CHUNK IS: ['asistencia técnica para soporte experto legal y financiero en el proyecto uia01-060, as-fabrik']
DETECTED ACR IN CHUNK BY LLAMA: ['as-fabrik']
Los detected (1) ['as-fabrik']
Los detected (2) ['as-fabrik']
Los detected (3) ['as-fabrik']
Response: true
VALIDATOR SAYS THAT 'as-fabrik' IT'S: true
EL ACRONYM ES as-fabrik
El NUEVO ID ES: PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_5.atom/149_0
EL ACRONYM ES: as-fabrik
Database loaded for document ID PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_5.atom/149_0 with metadata: {'index_aux': 'PlataformasAgregadasSinM

DETECTED ACR IN CHUNK BY LLAMA: ['cd', ' actuaciones urgentes de medidas constructivas y de reparación en las instalaciones del conde-duque', ' acuerdo marco de obras de reforma', ' reparación', ' restauración y rehabilitación del conjunto de monumentos y edificaciones', ' demaniales y patrimoniales competencia del área de gobierno de cultura y deport']
Los detected (1) ['restauración y rehabilitación del conjunto de monumentos y edificaciones', 'actuaciones urgentes de medidas constructivas y de reparación en las instalaciones del conde-duque', 'cd', 'reparación', 'acuerdo marco de obras de reforma', 'demaniales y patrimoniales competencia del área de gobierno de cultura y deport']
Los detected (2) ['cd', 'reparación']
Los detected (3) ['cd', 'reparación']
Response: false
VALIDATOR SAYS THAT 'cd' IT'S: false
Response: false
VALIDATOR SAYS THAT 'reparación' IT'S: false
PROCESSING ROW WITH TEXT:contrato basado 3.2 madrid destino.
PROCESSING CHUNK 0
CHUNK IS: ['contrato basado 3.2 madrid

DETECTED ACR IN CHUNK BY LLAMA: ['/']
Los detected (1) []
No acronyms detected in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_5.atom/13, continue the loop.
PROCESSING ROW WITH TEXT:dirección facultativa de las obras de construcción de 12 viviendas sociales en la planta primera de la calle txomin garat nº 7 de txurdinaga, bilbao (bizkaia).
PROCESSING CHUNK 0
CHUNK IS: ['dirección facultativa de las obras de construcción de 12 viviendas sociales en la planta primera de la calle txomin garat nº 7 de txurdinaga, bilbao (bizkaia).']
DETECTED ACR IN CHUNK BY LLAMA: ['/']
Los detected (1) []
No acronyms detected in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_5.atom/10, continue the loop.
PROCESSING ROW WITH TEXT:asistencia tecnica redacción de proyectos y direccion de obras: santurtzi, zorrotzaurre, mundaka
PROCESSING CHUNK 0
CHUNK IS: ['asistencia tecnica redacción de proyectos y direccion de obras:

DETECTED ACR IN CHUNK BY LLAMA: ['sac']
Los detected (1) ['sac']
Los detected (2) ['sac']
Los detected (3) []
No acronyms AFTER FILTERING in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_4.atom/469, continue the loop.
PROCESSING ROW WITH TEXT:actualización e implementación de las herramientas software del sistema integral de gestión de emergencias (sige) de la subdirección de protección civil.
PROCESSING CHUNK 0
CHUNK IS: ['actualización e implementación de las herramientas software del sistema integral de gestión de emergencias (sige) de la subdirección de protección civil.']
DETECTED ACR IN CHUNK BY LLAMA: ['sige', ' subdirección de protección civil', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' sige', ' s']
Los detected (1) ['sige', 's', 'subdirección de protección civil']
Los detected (2) ['sige', 's']


DETECTED ACR IN CHUNK BY LLAMA: ['atap', ' ap3e', ' santuxu']
Los detected (1) ['atap', 'ap3e', 'santuxu']
Los detected (2) ['atap', 'ap3e', 'santuxu']
Los detected (3) []
No acronyms AFTER FILTERING in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_4.atom/455, continue the loop.
PROCESSING ROW WITH TEXT:licitacion desierta: servicio de auditorias para la realización de las actuaciones de control financiero
PROCESSING CHUNK 0
CHUNK IS: ['licitacion desierta: servicio de auditorias para la realización de las actuaciones de control financiero']
DETECTED ACR IN CHUNK BY LLAMA: ['lda', ' sca', ' caf']
Los detected (1) ['caf', 'lda', 'sca']
Los detected (2) ['caf', 'lda', 'sca']
Los detected (3) []
No acronyms AFTER FILTERING in row PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_4.atom/454, continue the loop.
PROCESSING ROW WITH TEXT:servicio de elaboración del módulo de inventario de la estructura del servi

In [508]:
df_esp_updated

,text,idioma,Acronyms Validated,Not Acronyms,Is there any acronym
identifier,,,,,
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/453,refuerzo de firme en la vp 3001 renedo de esgu...,SPANISH,"pk, vp",,True
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/452,refuerzo de firme en la vp 6603 mota del marqu...,SPANISH,vp,"casasola, zamora, se, a-6, lp",True
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/451,refuerzo de firme en la vp 4013 melgar de arri...,SPANISH,,,False
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/448,desarrollo del programa de intervención socioe...,SPANISH,,,False
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/447,stand de euskadi en fitur y sus posibles adap...,SPANISH,fitur,"e, ferias, euskadi",True
...,...,...,...,...,...
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_4.atom/451,servicio de comunicación de las actividades de...,SPANISH,,,False
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_4.atom/450,servicio para la redacción del estudio hidroge...,SPANISH,,,False
PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_190110_4.atom/449,servicio del programa de intervención social c...,SPANISH,,,False


In [522]:
filtered_df = df_esp_updated[df_esp_updated['Acronyms Validated'].str.strip() != '']
filtered_df.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/detected_agosto.xlsx')

In [276]:
first_index_value

'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137_1.atom/453'

In [184]:
#!pip install lingua-language-detector

path = '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/DESCARGAS_ENTREGABLES/outsiders.parquet'
df = pd.read_parquet(path)

from lingua import Language, LanguageDetectorBuilder
# Usar detect_languages_in_parallel_of para procesar los títulos
# Column text must be provided in df
df.rename(columns={'title': 'text'}, inplace=True)

languages = [Language.ENGLISH, Language.BASQUE, Language.CATALAN, Language.SPANISH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
idiomas_detectados = detector.detect_languages_in_parallel_of(df['text'].tolist())

# Añadir los resultados como una nueva columna en el DataFrame
df['idioma'] = [idioma.name if idioma else 'Indefinido' for idioma in idiomas_detectados]

df = df[['text','idioma']]

df_esp = df[df['idioma'] == 'SPANISH']

def unify_colname(col):
    return ".".join([el for el in col if el])

df_esp.columns = [unify_colname(col) for col in df_esp.columns]
print(len(df_esp))
print(df_esp.columns)
#Para coger ['zip', 'file name', 'entry']
index_names = df_esp.index.names
print(index_names)
#Se resetea el índice sobre el propio dataframe
df_esp.reset_index(inplace=True)
df_esp['identifier'] = df_esp[index_names].astype(str).agg("/".join, axis=1)
df_esp = df_esp[['text','idioma','identifier']]
# Cojo las X primeras filas
df_esp = df_esp.head(1000)

153061
Index(['text', 'idioma'], dtype='object')
['zip', 'file name', 'entry']


/tmp/ipykernel_362135/3566568876.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_esp['identifier'] = df_esp[index_names].astype(str).agg("/".join, axis=1)


In [185]:
df_esp

,text,idioma,identifier
0,Refuerzo de Firme en la VP 3001 Renedo de Esgu...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
1,Refuerzo de Firme en la VP 6603 Mota del Marqu...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
2,Refuerzo de firme en la VP 4013 Melgar de Arri...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
3,Desarrollo del programa de intervención socioe...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
4,STAND DE EUSKADI EN FITUR Y SUS POSIBLES ADAP...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
...,...,...,...
995,Servicio de comunicación de las actividades de...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
996,Servicio para la redacción del estudio hidroge...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
997,Servicio del programa de Intervención Social c...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...
998,Programa municipal denominado Punto de Informa...,SPANISH,PlataformasAgregadasSinMenores_2018.zip/Plataf...


In [740]:
process_dataframe_and_update_db(df_esp, collection)

Processing row ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 453): refuerzo de firme en la vp 3001 renedo de esgueva a pesquera de duero. tramo: villabañez a pk 12+422
Processing chunk 0
Detected acronyms in chunk: ['vp', ' pk']
EL ACRONYM ES vp
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 453)_ida0_0
EL ACRONYM ES: vp
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 453)_ida0_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 453)_ida0_0", 'acr': 'vp'}
EL ACRONYM ES pk
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 453)_ida1_0
EL ACRONYM ES: pk
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'Platafor

Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 420)_ida0_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 420)_ida0_0", 'acr': 'pnps'}
EL ACRONYM ES sste
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 420)_ida1_0
EL ACRONYM ES: sste
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 420)_ida1_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 420)_ida1_0", 'acr': 'sste'}
EL ACRONYM ES ct
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 420)_ida2_0
EL ACRONYM ES: ct
Database loaded for document ID ('PlataformasAgregadasSinM

Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 402)_ida1_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 402)_ida1_0", 'acr': 'gocd'}
Processing row ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 401): arrendamiento, sin opción a compra e incluyendo el mantenimiento, del material deportivo cardiovascular y de musculación para los centros deportivos municipales arganzuela, centro integrado y marqués de samaranch en el distrito de arganzuela.
Processing chunk 0
Detected acronyms in chunk: ['arganzuela', ' samaranch']
EL ACRONYM ES arganzuela
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 401)_ida0_0
EL ACRONYM ES: arganzuela
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip'

Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 397)_ida0_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 397)_ida0_0", 'acr': 'atm'}
EL ACRONYM ES madrid
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 397)_ida1_0
EL ACRONYM ES: madrid
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 397)_ida1_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 397)_ida1_0", 'acr': 'madrid'}
Processing row ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 394): contratación de una póliza de seguro de vida del personal empleado del centro de cálculo de ála

Detected acronyms in chunk: ['ocyt', ' cm']
EL ACRONYM ES ocyt
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 277)_ida0_0
EL ACRONYM ES: ocyt
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 277)_ida0_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 277)_ida0_0", 'acr': 'ocyt'}
EL ACRONYM ES cm
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 277)_ida1_0
EL ACRONYM ES: cm
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 277)_ida1_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 277)_ida1_0", 'acr': 'cm'}
Processing row (

Detected acronyms in chunk: ['e4', ' auditorio']
EL ACRONYM ES e4
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 261)_ida0_0
EL ACRONYM ES: e4
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 261)_ida0_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 261)_ida0_0", 'acr': 'e4'}
EL ACRONYM ES auditorio
El NUEVO ID ES: ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 261)_ida1_0
EL ACRONYM ES: auditorio
Database loaded for document ID ('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 261)_ida1_0 with metadata: {'index_aux': "('PlataformasAgregadasSinMenores_2018.zip', 'PlataformasAgregadasSinMenores_20180217_180137_1.atom', 261)_ida1_0", 'acr': 'auditori

In [ ]:
entry_id = '21_ida1_0'
bbdd = collection.get(
    ids = entry_id
)
bbdd

In [240]:
bbdd = collection.get(limit =100,include = ['documents','metadatas'])
bbdd

{'ids': ['PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/10_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/115_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/116_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/117_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/118_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/119_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/156_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/157_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/158_0',
  'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinM

In [241]:
all_ids = []
limit = 100
offset = 0

# Fetch all IDs from the collection in batches
while True:
    results = collection.get(limit=limit, offset=offset, include=['metadatas'])
    if not results['metadatas']:
        break
    all_ids.extend(results['metadatas'])
    offset += limit

In [675]:
#df1.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/results_first_process.xlsx')

In [676]:
df1

,idDB,document,context,acr,expansion,text_with_expansions
0,0_ida0_0,la ong ha lanzado una campaña de concienciació...,la ong ha lanzado una campaña de concienciació...,ong,organización no gubernamental,la organización no gubernamental ha lanzado un...
1,10_ida0_0,la programación de televisión española ha sido...,la programación de televisión española ha sido...,tve,televisión española,la programación de televisión española ha sido...
2,11_ida0_0,la empresa ha decidido cambiar su estructura l...,la empresa ha decidido cambiar su estructura l...,s.a.,sociedad anónima,la empresa ha decidido cambiar su estructura l...
3,12_ida0_0,el vicepresidente vp de la empresa presentó el...,el vicepresidente vp de la empresa presentó el...,vp,vicepresidente,el vicepresidente vicepresidente de la empresa...
4,13_ida0_0,el accidente ocurrió en el pk 50 de la autopis...,el accidente ocurrió en el pk 50 de la autopis...,pk,punto kilométrico,el accidente ocurrió en el punto kilométrico 5...
5,14_ida0_0,la ley de apoyo a las pyme ha sido aprobada po...,la ley de apoyo a las pyme ha sido aprobada po...,pib,producto interno bruto,la ley de apoyo a las pyme ha sido aprobada po...
6,14_ida1_0,la ley de apoyo a las pyme ha sido aprobada po...,la ley de apoyo a las pyme ha sido aprobada po...,pyme,pequeñas y medianas empresas,la ley de apoyo a las pequeñas y medianas empr...
7,15_ida0_0,las diferentes ccaa han implementado medidas p...,las diferentes ccaa han implementado medidas p...,ccaa,comunidades autónomas,las diferentes comunidades autónomas han imple...
8,15_ida1_0,las diferentes ccaa han implementado medidas p...,las diferentes ccaa han implementado medidas p...,virus,venereal infection and respiratory syndrome,las diferentes ccaa han implementado medidas p...
9,16_ida0_0,la comisión nacional del mercado de valores (c...,la comisión nacional del mercado de valores (c...,cnmv,comisión nacional del mercado de valores,la comisión nacional del mercado de valores (c...


In [468]:
#acronym_retriever = AcronymAwareRetriever(vectordb=collection, k=5) No lo uso al final
acronym_expander = AcronymExpanderModule()

In [493]:
def extract_passages(text, acronym):
        """
        Extract the two preceding and two following sentences after the first appearance of the acronym in
        the text.
        """
        sentences = sent_tokenize(text)
        for i, sentence in enumerate(sentences):
            if acronym.lower() in sentence.lower():
                start = max(i - 2, 0)
                end = min(i + 3, len(sentences))
                return ' '.join(sentences[start:end])
        return ""

In [664]:
def disambiguation_in_text(text, acronym, expansion):
    """
    This function locates all occurrences of the acronym in the text and replaces them with the expansion.
    
    text: The text in which the replacement should be made.
    acronym: The acronym to search for in the text.
    expansion: The expansion that will replace the acronym in the text.
    """
    # Regex pattern to find the acronym
    pattern = re.compile(r'\b' + re.escape(acronym) + r'\b')
    
    # Replace all occurrences of the acronym with the expansion
    result_text = pattern.sub(expansion, text)
    
    return result_text 

In [741]:
# 2nd Given the collection, apply Expansion, update collection and create dataframe with relevant information. 
def process_and_update_expansion(collection):
    
    all_ids = []
    limit = 1
    offset = 0

    # Fetch all IDs from the collection in batches
    while True:
        results = collection.get(limit=limit, offset=offset, include=['metadatas'])
        if not results['metadatas']:
            break
        all_ids.extend(results['metadatas'])
        offset += limit
    
    # Storage the dataFrame info that will be created
    data = []
            
    # Iterate through each metadata entry
    for metadata in all_ids:
        entry_id = metadata['index_aux']
        print("El entry_id es:", entry_id)
        
        # Fetch entry details from the collection
        entry = collection.get(where={"index_aux": {"$eq": entry_id}}, include=['documents', 'metadatas'])
        documents = entry.get('documents', [])
        metadatas = entry.get('metadatas', []) 
        
        print("El document get es:\n", documents)
        print("Los metadata get son:\n", metadatas)
        
        # Iterate through each document and metadata pair
        for document, metadata in zip(documents, metadatas):
            acronym = metadata.get('acr')
            print("El ACRONYM de la bbdd con get es", acronym)
            if acronym:
                # Extract the context for the acronym
                context = extract_passages(document, acronym)
                if context == '':
                    context = 'None'
                    print("The detected acronym it is not in contained in the text, HALLUCINATION!")
                print("EL context del passages ES\n", context)

                index = metadata.get('index_aux')
                print("EL index es:\n", index)

                # Expand the acronym using the AcronymExpanderModule
                expansion_result = acronym_expander.forward(context, acronym)
                expansion = expansion_result.expansion.lower() if expansion_result else "None"
                expansion = clean_acronyms(expansion)
                print("La EXPansion es:\n", expansion)

                # Update the metadata fields with expansions and context
                metadata['expansion'] = expansion
                metadata['context'] = context
                
                text_with_expansions = disambiguation_in_text(document, acronym, expansion)

                # Update the collection with new metadata
                collection.update(
                    ids=[entry_id],
                    metadatas=[metadata]
                )

                # Create dataframe that contains infoabout datase_index, documents, context, acr, expansion
                data.append({
                    'idDB': index,
                    'document': document,
                    'context': context,
                    'acr': acronym,
                    'expansion': expansion,
                    'text_with_expansions':text_with_expansions
                })

    # Create DataFrame from data collected 
    df = pd.DataFrame(data, columns=['idDB', 'document', 'context', 'acr', 'expansion','text_with_expansions'])

    return df

In [ ]:
#df_pruebas = process_and_update_expansion(collection)

In [744]:
#df_pruebas.to_excel('/export/usuarios_ml4ds/cggamella/RAG_tool/files/results_outsiders_process.xlsx')

In [347]:
'''
def searchDataByVector(query: str):
    try:
        query_vector = embedding_model.encode(query).tolist()
        res = collection.query(
            query_embeddings=[query_vector],
            n_results=1,
            include=['distances','embeddings', 'documents'],
        )
        print("Query", "\n--------------")
        print(query)
        print("Result", "\n--------------")
        print(res['documents'][0][0])
        print("Vector", "\n--------------")
        print(res['embeddings'][0][0])
        print("")
        print("")
        print("Complete Response","\n-------------------------")
        print(res)

    except Exception as e:
        print("Vector search failed : ", e)
'''

In [348]:
example_pdf_1 = "El PDF, formato de documento portátil, se ha convertido en un estándar para la distribución de documentos en línea. Permite a los usuarios compartir información de manera segura y profesional, preservando el formato original independientemente del software o dispositivo utilizado para su visualización."
example_pdf_2 = "En el campo de la ingeniería, es crucial para evaluar la fiabilidad de estructuras y materiales. Ayuda a los ingenieros a modelar y prever el comportamiento de sistemas complejos mediante el análisis de la distribución de probabilidades. Desde la planificación de proyectos hasta la gestión de riesgos, la PDF proporciona herramientas poderosas para optimizar el diseño y la operación de infraestructuras críticas."

searchDataByVector(query="formato de documento portátil")

Query 
--------------
formato de documento portátil
Result 
--------------
el ceip (centro de educación infantil y primaria) de la localidad ha implementado un nuevo programa de actividades extracurriculares para fomentar el desarrollo integral de los alumnos. los padres y madres de los estudiantes han mostrado gran interés en estas iniciativas, que van desde talleres de arte hasta clases de idiomas. el ceip se ha convertido en un referente en la comunidad por su enfoque innovador en la educación de los más pequeños.
Vector 
--------------
[0.02481706626713276, 0.021999306976795197, -0.030746091157197952, -0.012899807654321194, -0.09807442128658295, 0.006474603433161974, -0.1272658109664917, -0.10558166354894638, -0.034650158137083054, 0.19425718486309052, 0.19402752816677094, 0.1965489685535431, 0.06693072617053986, 0.07718990743160248, 0.09038682281970978, 0.05886144936084747, 0.01722206175327301, 0.030920827761292458, 0.03983842208981514, -0.028650298714637756, -0.04399373382329941,

# Custom Retrieval Module

In [390]:
class AcronymAwareRetriever(dspy.Retrieve):
    """
    Custom retriever for searching documents containing a specific acronym.
    """
    def __init__(self, vectordb, k:int = 3):
        super().__init__(k=k)
        self.vectordb = vectordb

    def forward(self, acronym: Union[str, List[str]], k: int = None) -> dspy.Prediction:
        """
        Retrieve documents containing the provided acronym by directly querying
        the vector database. The documents are filtered to include only those
        where the acronym appears as a standalone word.
        """
        # Creating regex to match the acronym as a standalone word
        regex = r'\b' + re.escape(acronym) + r'\b'

        # Retrieve k documents that might contain the acronym
        #results = self.vectordb.get(where_document={"$contains": acronym}, limit=k if k else self.k,
        #                            include=['documents', 'embeddings', 'metadatas'])
        metadata_filter = {
            "acr": {
                "$eq": acronym
            }
        }
        # Retrieve k documents which had the acronym as metadata
        results = self.vectordb.get(where=metadata_filter, limit=k if k else self.k,
                                    include=['documents', 'embeddings', 'metadatas'])

        # Extract relevant information
        documents = results.get('documents', [])
        embeddings = results.get('embeddings', [])
        metadatas = results.get('metadatas', {})

         # Filter documents and their embeddings using regex, avoiding duplicates
        seen_documents = set()
        filtered_documents = []
        filtered_embeddings = []
        filtered_passages = []
        filtered_metadatas = []
        
        # Create a mapping of documents to their corresponding metadatas
        document_to_metadata = dict(zip(documents, metadatas))

        for doc, emb in zip(documents, embeddings):
            # Check if the document contains the acronym as a standalone word and is not already added
            if re.search(regex, doc, re.IGNORECASE) and doc not in seen_documents:
                seen_documents.add(doc)
                filtered_documents.append(doc)
                filtered_embeddings.append(emb)
                # Extract passages for documents that pass the regex check
                filtered_passages.append(self.extract_passages(doc, acronym))
                # Append corresponding metadata
                filtered_metadatas.append(document_to_metadata.get(doc, {}))

        return dspy.Prediction(
            metadatas = filtered_metadatas,
            documents=filtered_documents,
            embeddings=filtered_embeddings,
            passages=filtered_passages
        )

    def extract_passages(self, text, acronym):
        """
        Extract the two preceding and two following sentences after the first appearance of the acronym in
        the text.
        """
        sentences = sent_tokenize(text)
        for i, sentence in enumerate(sentences):
            if acronym.lower() in sentence.lower():
                start = max(i - 2, 0)
                end = min(i + 3, len(sentences))
                return ' '.join(sentences[start:end])
        return ""

In [ ]:
# Crear instancia del retriever
retriever = AcronymAwareRetriever(vectordb=collection)  
acronym = "ue"

# Realizar la búsqueda
prediction = retriever.forward(acronym=acronym, k=1)
prediction

# Desambiguación de acrónimos con dspy (Etapa 2)

In [848]:
class AcronymExpander(dspy.Signature):
    """
    Expande los acrónimos basándose en el contexto del texto.
    """
    TEXTO = dspy.InputField(desc="Texto que contiene el acrónimo,sigla o abreviatura")
    ACRONIMO = dspy.InputField()
    EXPANSION = dspy.OutputField()

class AcronymExpanderModule(dspy.Module):
    def __init__(self):
        super().__init__()
        #self.expander = dspy.Predict(AcronymExpander)
        self.expander = dspy.ChainOfThought(AcronymExpander)

    def forward(self, texto, acronimo):
        response = None
        try:
            response = self.expander(TEXTO=texto, ACRONIMO=acronimo)
            print(f"Estás en el forward la respuesta para el texto '{texto}' es:{response}")
        except Exception as e:
            print(f"-- -- Error expanding acronym: {e}")
            response = None

        return dspy.Prediction(
            texto=texto,
            acronimo=acronimo,
            expansion=response.EXPANSION if response else None
        )

In [832]:
texto="Ejemplo de texto para pib de la sociedad española de la federación de empresarios agrarios"
acronimo="agrarios"
# Uso del módulo
module = AcronymExpanderModule()
result = module(texto,acronimo)

Estás en el forward la respuesta para el texto 'Ejemplo de texto para pib de la sociedad española de la federación de empresarios agrarios' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Asociación de Ganaderos y Agricultores de la Región de Andalucía'
)
En el forward la respuesta para el texto 'Ejemplo de texto para pib de la sociedad española de la federación de empresarios agrarios' es: Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Asociación de Ganaderos y Agricultores de la Región de Andalucía'
)
Rationale completo: produce the EXPANSION. We...


# Disambiguation logic

In [78]:
vectordb=collection

In [467]:
acronym_retriever = AcronymAwareRetriever(vectordb=collection, k=5)
acronym_expander = AcronymExpanderModule()

In [ ]:
'''
# Creo nueva columna en el df para guardar los acrónimos expandidos
df['expanded_LLaMA'] = 'None'

for index, row in df.iterrows():
    text = row['text']
    # Los acrónimos están separados por comas si se detecta más de uno.
    detected_acronyms = str(row['detected_acronyms_LLaMA']).lower().split(',')
    print(detected_acronyms)
    detected_acronyms = [acronym.strip() for acronym in detected_acronyms if acronym.strip() not in ['', 'n/a', 'none', 'ninguno']]

    # Lista para expansiones de acrónimos encontrados
    expansions = []

    for acronym in detected_acronyms:
        # Recuperar documentos relevantes utilizando el retriever
        retrieved_docs = acronym_retriever.forward(acronym).passages

        # Concatenar todos los docs retrieval
        concatenated_docs = " ".join(retrieved_docs)
        # Usar docs concatenados
        if concatenated_docs:
            expansion_result = acronym_expander.forward(concatenated_docs, acronym)
            expansions.append(expansion_result.expansion)

    # Guardar las expansiones en la columna 'expanded_LLaMA'
    df.at[index, 'expanded_LLaMA'] = ', '.join(expansions)
'''

# Integración de grafos de conocimiento para desambiguación de contextos

In [ ]:
'''
class GraphVisualizer:
    def __init__(self, kernel='umbral', threshold=0.21, remove_self_links=True):
        self.kernel = kernel
        self.threshold = threshold
        #self.symmetric = symmetric
        self.remove_self_links = remove_self_links

    def set_embeddings(self, embeddings):
        self.embeddings = np.array(embeddings)

    def _get_similarity_matrix(self):
        if self.embeddings is None:
            raise ValueError("Please set embeddings before trying to get similarity matrix.")
        #print(type(self.embeddings), self.embeddings.shape)
        print(self.embeddings)
        similarity_matrix = cosine_similarity(self.embeddings)
        print("La matriz de similitud es:\n", similarity_matrix)
        if self.kernel == "umbral":
            similarity_matrix[similarity_matrix < self.threshold] = 0
        if self.remove_self_links:
            np.fill_diagonal(similarity_matrix, 0)
        return similarity_matrix

    def visualize_graph(self, plot_graph=True):
        similarity_matrix = self._get_similarity_matrix()
        G = nx.from_numpy_array(similarity_matrix, create_using=nx.DiGraph)

        if plot_graph:
            pos = nx.spring_layout(G, seed=42)
            nx.draw(G, pos, node_color='skyblue', with_labels=True, node_size=70, edge_color='blue', width=0.4)
            plt.title("Document Similarity Graph")
            plt.show()

        return {
            'number_of_nodes': G.number_of_nodes(),
            'number_of_edges': G.number_of_edges(),
            'edges_per_node': G.number_of_edges() / G.number_of_nodes()
        }
'''

In [ ]:
'''
result_predict = acronym_retriever.forward("pdf")

# Instanciar y configurar GraphVisualizer
visualizer = GraphVisualizer(kernel='umbral', threshold=0.31, remove_self_links=True)
visualizer.set_embeddings(result_predict.embeddings)

# Visualizar el grafo de similitud
visualizer.visualize_graph()
'''

# Crear dataset de entrenamiento y validación para DSPY

In [623]:
'''
def create_dtset_expanded_acronyms(excel_path):
    # Cargar datos
    df = pd.read_excel(excel_path)
    df = df[['text', 'detected_acr', 'acr_des']]

    # Dividir los datos en conjuntos de entrenamiento y validación
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

    # Convertir los df de train y dev en listas de diccionarios
    data_train = train_df.to_dict('records')
    data_test = test_df.to_dict('records')

    # Create train examples
    trainset = [
        dspy.Example({'texto': row['text'], 'acronimos': row['detected_acr'], 'expansion': row['acr_des']}) for row in data_train
    ]
    # Create dev examples
    devset = [
        dspy.Example({'texto': row['text'], 'acronimos': row['detected_acr'], 'expansion': row['acr_des']}) for row in data_test
    ]

    return trainset, devset
'''

In [724]:
def create_dtset_detected_acronyms(excel_path):
    # Cargar datos
    df = pd.read_excel(excel_path)
    df = df[['text', 'detected_acr', 'acr_des']]

    # Dividir los datos en conjuntos de entrenamiento y validación
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

    # Convertir los df de train y dev en listas de diccionarios
    data_train = train_df.to_dict('records')
    data_test = test_df.to_dict('records')

    # Crear ejemplos de entrenamiento con inputs configurados
    trainset = [
        dspy.Example({'texto': row['text'], 'detected_acronimos': row['detected_acr'], 'expansion': row['acr_des']})
        .with_inputs('texto') for row in data_train
    ]
    
    # Crear ejemplos de validación con inputs configurados
    devset = [
        dspy.Example({'texto': row['text'], 'detected_acronimos': row['detected_acr'], 'expansion': row['acr_des']})
        .with_inputs('texto') for row in data_test
    ]

    return trainset, devset

In [760]:
def create_dtset_expanded_acronyms(excel_path):
    # Cargar datos
    df = pd.read_excel(excel_path)
    df = df[['text', 'detected_acr', 'acr_des']]
    df = df[df['detected_acr'] != '/']
    print(len(df))

    # Dividir los datos en conjuntos de entrenamiento y validación
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

    # Convertir los df de train y dev en listas de diccionarios
    data_train = train_df.to_dict('records')
    data_test = test_df.to_dict('records')

    # Crear ejemplos de entrenamiento con inputs configurados
    trainset = [
        dspy.Example({'texto': row['text'], 'acronimo': row['detected_acr'], 'expansion': row['acr_des']})
        .with_inputs('texto','acronimo') for row in data_train
    ]
    
    # Crear ejemplos de validación con inputs configurados
    devset = [
        dspy.Example({'texto': row['text'], 'acronimo': row['detected_acr'], 'expansion': row['acr_des']})
        .with_inputs('texto','acronimo') for row in data_test
    ]

    return trainset, devset

In [406]:
'''
excel_path = '/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/fam/df_esp_100_200.xlsx'
# Cargar los datos
df = pd.read_excel(excel_path)
df = df[['text','manual_expanded_acronyms', 'detected_acronyms_LLaMA','expanded_LLaMA']]

# Dividir los datos en entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

def create_examples(df):
    examples = []
    for index, row in df.iterrows():
        # Asegurar que ambos campos, acrónimos y expansiones, contienen datos
        if pd.notna(row['detected_acronyms_LLaMA']) and pd.notna(row['expanded_LLaMA']):
            acronyms_list = [acr.strip() for acr in row['detected_acronyms_LLaMA'].split(',')]
            expansions_list = [exp.strip() for exp in row['expanded_LLaMA'].split(',')]

            # Verificar que ambos listados tengan el mismo número de elementos
            if len(acronyms_list) != len(expansions_list):
                print(f"Error en la fila {index}: Número desigual de acrónimos y expansiones.")
                print(f"Fila problemática: {row.to_dict()}")
                continue

            # Crear un ejemplo por cada acrónimo y su correspondiente expansión
            for acr, exp in zip(acronyms_list, expansions_list):
                examples.append(dspy.Example({
                    'texto': row['text'],
                    'acronimo': acr,
                    'expansion_correcta': exp
                }).with_inputs('texto', 'acronimo'))
        else:
            print(f"Datos faltantes en la fila {index}.")
            print(f"Fila problemática: {row.to_dict()}")

    return examples


train_examples = create_examples(train_df)
test_examples = create_examples(test_df)
train_examples
print(len(test_examples))
print(len(train_examples))
'''

In [626]:
'''
# Cargar el modelo de SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def validate_expansion_similarity(example, pred, trace = None):

    prediction_embedding = model.encode(pred.expansion)
    reference_embedding = model.encode(example.expansion_correcta)

    print("La pred embedding es:", prediction_embedding)
    print("La ref embedding es:", reference_embedding)

    # Calcular la similitud del coseno
    similarity = cosine_similarity([prediction_embedding], [reference_embedding])[0][0]

    # Evaluar si la similitud está por encima de un umbral-> [-1,1]
    if similarity > 0.9:
        # Considerando que si está por encima de 0.9 es un acierto
        print(similarity)
        return True
    else:
        print(similarity)
        return False
    

# Testing validate_expansion metric
example = type('Example', (object,), {'expansion_correcta': "El PDF, formato de documento portátil, se ha convertido en un estándar para la distribución de documentos en línea. Permite a los usuarios compartir información de manera segura y profesional, preservando el formato original independientemente del software o dispositivo utilizado para su visualización."})
pred = type('Prediction', (object,), {'expansion': "En el campo de la ingeniería, es crucial para evaluar la fiabilidad de estructuras y materiales. Ayuda a los ingenieros a modelar y prever el comportamiento de sistemas complejos mediante el análisis de la distribución de probabilidades. Desde la planificación de proyectos hasta la gestión de riesgos, la PDF proporciona herramientas poderosas para optimizar el diseño y la operación de infraestructuras críticas."})
is_correct = validate_expansion_similarity(example, pred)
print("¿Es correcta la expansión?:", is_correct)
'''

In [745]:
def validate_acronym_detection(example, pred, trace=None):
    """
    Valida si el acrónimo predicho está presente en el texto de example.

    example: Objeto que contiene la expansión correcta (texto).
    pred: Objeto Prediction que contiene el acrónimo predicho.
    trace: Parámetro opcional para trazabilidad.
    
    return: 1 si el acrónimo está presente en el texto o si se predice '/' cuando no hay acrónimos, 0 en caso contrario.
    """
    # Convertimos el texto y el acrónimo predicho a minúsculas para comparación insensible a mayúsculas/minúsculas
    text_lower = example['texto'].lower()
    pred_acronym_lower = pred.ACRONIMOS.lower()

    # Si el modelo devuelve '/', valido si no había acrónimos en el texto original
    if pred_acronym_lower == '/':
        # Asumimos que '/' es correcto si no hay acrónimos en el texto
        if example['detected_acronimos'] == '/':  
            print(f"Correctamente no se detectaron acrónimos y se devolvió '/'.")
            return 1
        else:
            print(f"Error: Se devolvió '/' pero sí hay acrónimos en el texto. En este caso hay: {example['detected_acronimos']}")
            return 0

    if pred_acronym_lower in text_lower:
        print(f"El acrónimo '{pred_acronym_lower}' está presente en el texto.")
        return 1
    else:
        print(f"El acrónimo '{pred_acronym_lower}' no está presente en el texto.")
        return 0

In [841]:
def validate_expansion_acronym(example, pred, trace=None):
    """
    Valida si la expansión de un acrónimo predicho es correcta en el contexto del texto dado.

    example: Objeto que contiene el texto, acrónimos detectados, y la expansión correcta esperada.
    pred: Objeto Prediction que contiene la expansión del acrónimo predicha por el modelo.
    trace: Parámetro opcional para trazabilidad.
    
    return: 1 si la expansión predicha es correcta, 0 en caso contrario.
    """
    # Convertimos el texto y la expansión predicha a minúsculas 
    text = example['texto'].lower()
    expected_expansion = example['expansion'].lower()
    acr = example['acronimo'].lower()
    
    # Verificamos si pred.expansion no es None
    if pred.expansion is None:
        print("ERROR: La expansión predicha es None.")
        return 0
    
    pred_expansion = pred.expansion.lower()
    print("El texto es", text)
    print("El acrónimo es", acr)
    print("La expansión esperada es:", expected_expansion)
    
    # Separar las palabras en listas
    expected_words = expected_expansion.split()
    pred_words = pred_expansion.split()

    # Contar el número de palabras coincidentes
    matching_words = sum(1 for word in pred_words if word in expected_words)
    matching_ratio = matching_words / len(expected_words)

    print(f"Palabras coincidentes: {matching_words}/{len(expected_words)} ({matching_ratio*100:.1f}%)")

    # Considerar un acierto si al menos la mitad de las palabras coinciden
    if matching_ratio >= 0.5:
        print(f"ACIERTO: La expansión predicha '{pred_expansion}' tiene suficientes coincidencias.")
        return 1
    else:
        print(f"ERROR: La expansión predicha '{pred_expansion}' no tiene suficientes coincidencias con '{expected_expansion}'.")
        return 0

    if pred_expansion == acr:
        print(f"La expansión predicha '{pred_expansion}' es igual que el candidato a expandir!.")
        return 0
    if pred_expansion == expected_expansion:
        print(f"ACIERTO: La expansión predicha '{pred_expansion}' es correcta.")
        return 0
    # Verificamos si la expansión predicha coincide con la expansión esperada
    if pred_expansion in expected_expansion:
        print(f"ACIERTO: La expansión predicha '{pred_expansion}' es correcta.")
        return 1
    else:
        print(f"ERROR: La expansión predicha '{pred_expansion}' no coincide con la expansión esperada '{expected_expansion}'.")
        return 0

In [842]:
class Prediction:
    def __init__(self, ACRONIMOS):
        self.ACRONIMOS = ACRONIMOS

example1 = {
    'texto': 'Elaboración del estudio ambiental de la zona costera de Monpas y de los estuarios de los ríos Oiartzun y Urumea',
    'detected_acronimos': '/'
}

pred1 = Prediction(ACRONIMOS='/')

# Debería devolver 1 ya que correctamente no se detectaron acrónimos y se devolvió '/'
result1 = validate_expansion(example1, pred1)
print(result1)

Correctamente no se detectaron acrónimos y se devolvió '/'.
1


In [843]:
excel_path = '/export/usuarios_ml4ds/cggamella/RAG_tool/files/anotacion_manual/fam/df_esp_100_200.xlsx'
# Crear conjuntos de entrenamiento y validación
trainset, devset = create_dtset_expanded_acronyms(excel_path)

37


In [844]:
trainset[0]

Example({'texto': '103/2017/05179. IDM ESTANQUE DEL RETIRO: Obras de acondicionamiento y reforma.', 'acronimo': 'IDM', 'expansion': 'iniciativa del desarrollo y modernización'}) (input_keys={'texto', 'acronimo'})

In [849]:
teleprompter = BootstrapFewShotWithRandomSearch(
    metric=validate_expansion_acronym,
    max_bootstrapped_demos=4,
    max_labeled_demos=16,
    num_candidate_programs=16,
    max_rounds=1,
)

In [836]:
print(len(trainset))
print(len(devset))

25
12


In [837]:
trainset[1]

Example({'texto': 'Adquisición de un equipo portátil RX para el Hospital Universitario La Paz', 'acronimo': 'RX', 'expansion': 'rayos x'}) (input_keys={'texto', 'acronimo'})

In [838]:
devset[0]

Example({'texto': 'Servicios de traslados, depósito, custodia y gestión de archivo de los fondos documentales de la Empresa Municipal de Vivienda y Suelo de Madrid, S.A.', 'acronimo': 'S.A.', 'expansion': 'sociedad anónima'}) (input_keys={'texto', 'acronimo'})

In [850]:
my_acronym_detector = AcronymDetectorModule()
my_acronym_expander = AcronymExpanderModule()

In [851]:
compiled_model = teleprompter.compile(
    my_acronym_expander,
    trainset=trainset,
    valset=devset
)

Average Metric: 11 / 12  (91.7): 100%|█████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 170.37it/s]


Estás en el forward la respuesta para el texto 'Servicios de traslados, depósito, custodia y gestión de archivo de los fondos documentales de la Empresa Municipal de Vivienda y Suelo de Madrid, S.A.' es:Prediction(
    rationale='${produce the EXPANSION}. We...',
    EXPANSION='Sociedad Anónima'
)
El texto es servicios de traslados, depósito, custodia y gestión de archivo de los fondos documentales de la empresa municipal de vivienda y suelo de madrid, s.a.
El acrónimo es s.a.
La expansión esperada es: sociedad anónima
Palabras coincidentes: 2/2 (100.0%)
ACIERTO: La expansión predicha 'sociedad anónima' tiene suficientes coincidencias.
Estás en el forward la respuesta para el texto 'Servicios para el seguimiento del proceso de sostenibilidad de Errenteria: Agenda Local 21 y Secretaría Técnica del Consejo Asesor de Medio Ambiente (CAMA) ' es:Prediction(
    rationale='${produce the EXPANSION}. We...',
    EXPANSION='Consejo Asesor de Medio Ambiente'
)
El texto es servicios para el segui

Average Metric: 1 / 1  (100.0):   0%|                                                                    | 0/12 [00:00<?, ?it/s]

Estás en el forward la respuesta para el texto 'Colector de Saneamiento del Sur de las Librilleras, KM 15, Los Curas y Otras en Lorca (Murcia)' es:Prediction(
    rationale='${produce the EXPANSION}. We...',
    EXPANSION='Kilómetro'
)
El texto es colector de saneamiento del sur de las librilleras, km 15, los curas y otras en lorca (murcia)
El acrónimo es km
La expansión esperada es: kilómetro
Palabras coincidentes: 1/1 (100.0%)
ACIERTO: La expansión predicha 'kilómetro' tiene suficientes coincidencias.
Estás en el forward la respuesta para el texto 'Aliviadero Colector General de Caravaca en Paraje Cañada de Lentisco, T.M. Caravaca de la Cruz (Murcia)' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Término Municipal'
)
El texto es aliviadero colector general de caravaca en paraje cañada de lentisco, t.m. caravaca de la cruz (murcia)
El acrónimo es t.m.
La expansión esperada es: término municipal
Palabras coincidentes: 2/2 (100.0%)
ACIERTO: La expansión pre

Average Metric: 11 / 12  (91.7): 100%|█████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 228.73it/s]


...

Estás en el forward la respuesta para el texto 'Aliviadero Colector General de Caravaca en Paraje Cañada de Lentisco, T.M. Caravaca de la Cruz (Murcia)' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Territorio Municipal'
)
El texto es aliviadero colector general de caravaca en paraje cañada de lentisco, t.m. caravaca de la cruz (murcia)
El acrónimo es t.m.
La expansión esperada es: término municipal
Palabras coincidentes: 1/2 (50.0%)
ACIERTO: La expansión predicha 'territorio municipal' tiene suficientes coincidencias.
Estás en el forward la respuesta para el texto 'Reforma fachadas C.C. Eduardo Úrculo. Distrito Tetuán' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Centro Comercial'
)
El texto es reforma fachadas c.c. eduardo úrculo. distrito tetuán
El acrónimo es c.c.
La expansión esperada es: centro comercial
Palabras coincidentes: 2/2 (100.0%)
ACIERTO: La expansión predicha 'centro comercial' tiene suficientes coincidencias

Average Metric: 9 / 11  (81.8):  83%|█████████████████████████████████████████████████▏         | 10/12 [00:01<00:00,  6.69it/s]

Estás en el forward la respuesta para el texto 'Construcción de intersección de la carretera RM-714 con camino cañada garcia P.K. 57+700 P.K. de Cehegin' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Kilómetro'
)
El texto es construcción de intersección de la carretera rm-714 con camino cañada garcia p.k. 57+700 p.k. de cehegin
El acrónimo es p.k.
La expansión esperada es: punto kilométrico 
Palabras coincidentes: 0/2 (0.0%)
ERROR: La expansión predicha 'kilómetro' no tiene suficientes coincidencias con 'punto kilométrico '.
Estás en el forward la respuesta para el texto 'Licitación para la contratación de suministro de plástico para INDESA 2010, S.L por procedimiento abierto y tramitación ordinaria.' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Sociedad Limitada'
)
El texto es licitación para la contratación de suministro de plástico para indesa 2010, s.l por procedimiento abierto y tramitación ordinaria.
El acrónimo es s.l


Average Metric: 10 / 12  (83.3): 100%|██████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.94it/s]

Estás en el forward la respuesta para el texto 'Contrato de Servicios Complementarios (mantenimiento, limpieza y conserjería) de los Edificios, Colegios e Instalaciones Deportivas dependientes del Distrito de Chamartín, 4 Lotes, (reservado Lote 4 a Empresas de Inserción, D.A. 5a del TRLCSP).' es:Prediction(
    rationale='produce the EXPANSION. We...',
    EXPANSION='Distrito Administrativo, Texto Refundido de la Ley de Contratos del Sector Público'
)
El texto es contrato de servicios complementarios (mantenimiento, limpieza y conserjería) de los edificios, colegios e instalaciones deportivas dependientes del distrito de chamartín, 4 lotes, (reservado lote 4 a empresas de inserción, d.a. 5a del trlcsp).
El acrónimo es d.a., trlcsp
La expansión esperada es: disposición adicional, texto refundido de la ley de contratos del sector público
Palabras coincidentes: 10/12 (83.3%)
ACIERTO: La expansión predicha 'distrito administrativo, texto refundido de la ley de contratos del sector público'

In [852]:
save_path = '/export/usuarios_ml4ds/cggamella/RAG_tool/files/model_dspy_expander.json'
compiled_model.save(save_path)